<a href="https://colab.research.google.com/github/dimitarpg13/langchain_tutorial/blob/main/langchain_tutorial/notebooks/runtime_config/SimpleRuntimeConfig.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with Runtime Configurations




In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_core langgraph langchain_openai